# Simulation - Part 2

In this document we will look at some more advanced animating options, the project structure (multi-step animations) and visualizing molecules originating from PDB-files.

The sections until assignment 5 can be used as reference; just read the parts that you think you need to complete the assignments.

## Table of Contents

Note: links do not work on the mirror

* [**Rendering a Specific Frame (number or time)**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Rendering-a-Specific-Frame-(number-or-time))

* [**Converting Frame Number to Time**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Converting-Frame-Number-to-Time)

* [**Visualizing Molecules**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Visualizing-Molecules)

    * [**Finding Molecules (PDB-files)**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Finding-Molecules-(PDB-files))

    * [**Placing Molecules**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Placing-Molecules)

    * [**Using a molecule in an animation**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Using-a-molecule-in-an-animation)

    * [**Molecule Movement**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Molecule-Movement)

    * [**Rotation**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Rotation)

    * [**Splitting Molecules**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Splitting-Molecules)

    * [**Styling**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Styling)

        * [**Atom Labels**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Atom-Labels)

* [**Simulation - Program Structure**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Simulation---Program-Structure)



* [**Assignment 5 - Molecular Structures**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Assignment-5---Molecular-Structures)

* [**Assignment 6 - Final Project Design**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Assignment-6---Final-Project-Design)

* [**Assignment 7 - Materials & Methods chapter**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/pypovray_simulation_2.ipynb#Assignment-6---Materials-&-Methods-chapter)



## Rendering a Specific Frame

When we create an animation that runs for 30 seconds at 25 frames per second there is a total of **750** frames to be rendered. This will take a long time to do while during development, you might be interested in only a few frames to check if objects are correctly positioned. In assignments **1** and **2** we created single-frames using the `render_scene_to_png` function:

    povray.render_scene_to_png(frame, 0)

We skipped explaining the `0`-argument given to this function which we can use to specify a specific *frame*. To render frame 30 we could simply use:

    povray.render_scene_to_png(frame, 30)

which creates a single image representing frame number 30 (i.e. the `frame` function is provided the integer `30` as the value of the `step` variable). This will become especially useful when we want to render a specific moment in our simultation without waiting for the whole process to finish. The frame number can *also* be specified as a `list` or `range()` object. The following example renders only the frames `72-93`:

    povray.render_scene_to_png(frame, range(72, 94))

## Converting Frame Number to Time

In the previous assignments we used the current frame (the `step` argument) to guide our animation. In the example we used this number to calculate if we were halfway through our animation to change the direction of the object. In this document we will create an animation consisting of multiple parts and you might want to keep track of the duration of each part in *seconds* instead of *frames*. This is just an alternative method which requires you to calculate where - in seconds - we are in the process. This is easy enough using the numeric values taken from the `SETTINGS` object which stores all content from your configuration file. The following line calculates *where* in the animation we are for the **current** step:

    timepoint = step / eval(SETTINGS.NumberFrames)

This is purely a guide-method that you might use if you'd rather think in seconds then in frame numbers. To use this calculation you can add the above and the following line to your `frame(step)` function to calculate and show (actually, *log*) where we are:

    logger.info("@Time: %.4fs, Step: %d", timepoint, step)

and the following line to your `main()` function to show how long it will take *in total*:

    logger.info(" Total time: %d (frames: %d)", SETTINGS.Duration, eval(SETTINGS.NumberFrames))

The following summarized example demonstrates this, note the extra import of the `SETTINGS` and `logger` objects:

In [8]:
#!/usr/bin/env python3
from pypovray import pypovray, models, logger, SETTINGS
from vapory import Sphere, Scene

def frame(step):
    ''' Creates a sphere and places this in a scene '''
    timepoint = step / eval(SETTINGS.NumberFrames)
    
    logger.info(" @Time: %.4fs, Step: %d", timepoint, step)
    
    sphere = Sphere([5, 2, 0], 3, models.default_sphere_model)

    # Return the Scene object for rendering
    return Scene(models.default_camera,
                 objects=[models.default_light, sphere])

if __name__ == '__main__':
    logger.info(" Total time: %ds (frames: %d)", SETTINGS.Duration, eval(SETTINGS.NumberFrames))
    
    # Render just the first 10 frames
    pypovray.render_scene_to_png(frame, range(10))

INFO:pypovray: Total time: 4 (frames: 80)
INFO:pypovray: @Time: 0.0000s, Step: 0
INFO:pypovray: @Time: 0.0125s, Step: 1
INFO:pypovray: @Time: 0.0250s, Step: 2
INFO:pypovray: @Time: 0.0375s, Step: 3
INFO:pypovray: @Time: 0.0500s, Step: 4
INFO:pypovray: @Time: 0.0625s, Step: 5
INFO:pypovray: @Time: 0.0750s, Step: 6
INFO:pypovray: @Time: 0.0875s, Step: 7
INFO:pypovray: @Time: 0.1000s, Step: 8
INFO:pypovray: @Time: 0.1125s, Step: 9


As you can see, at each call to our `frame` function, we print the actual time in seconds. Each frame takes 1/8th of a second as you can see by what we printed (0.0125 seconds) If we now want to do something from `0.5 .. 1.0` seconds we could write the following `if` statement:

    if 0.5 <= timepoint <= 1.0:
        ...

which checks if `timepoint` is *between* 0.5 and 1.0 (an *interval comparison*).

# Visualizing Molecules

While molecules can be represented with known objects such as `Spheres` and optionally `Cylinders`, manually visualizing complete proteins using these objects takes a lot of time. Therefore we offer a Python `pdb` module that draws complete molecules given a *Protein Data Bank* (PDB) file that should contain familiar contents. For instance, a simple water (H2O) molecule can be described in a PDB file with:

    ATOM      1  OH  OSP3    1       4.013   0.831  -9.083  1.00  0.00          O    
    ATOM      2 1HH  OSP3    1       4.941   0.844  -8.837  1.00  0.00          H    
    ATOM      3 2HH  OSP3    1       3.750  -0.068  -9.293  1.00  0.00          H    
    TER              

where each `ATOM` describes the specific atom and its location in 3D space. Such an `ATOM` 'record' in this file can contain [14 values](http://www.wwpdb.org/documentation/file-format-content/format33/sect9.html#ATOM) in *fixed-width* columns.  Optionally there can be connections defined that list connections between atoms for use in so called *ball-and-stick* models. Our module loads a PDB file and parses atoms and their position (for example the Oxygen atom is placed at `[`<font color='red'>4.013</font>`,` <font color='blue'>0.831</font>`,` <font color='green'>-9.083</font>`]`) which is defined in the following positions;

    31 - 38        Real(8.3)     x            Orthogonal coordinates for X in Angstroms.
    39 - 46        Real(8.3)     y            Orthogonal coordinates for Y in Angstroms.
    47 - 54        Real(8.3)     z            Orthogonal coordinates for Z in Angstroms.

As a user of our module you can easily load, place and orient complete molecules with a small set of simple commands which are all demonstrated below.

## Finding Molecules (PDB-files)

We are definately *not* going to create our own molecular structure (stored as PDB) files so we will use web-resources to gather these files when needed. Of course there is the central [*Protein Data Bank*](http://www.rcsb.org/pdb/home/home.do) (PDB) website but contents here are mostly limited to proteins (sometimes also containing its reactants). There are other web resources for non-protein molecules such as the [*World of Molecules*](http://www.worldofmolecules.com) and the [*Automated Topology Builder*](https://atb.uq.edu.au/index.py) (ATB). Performing a plain internet search combining your molecule of choice with the **PDB** term also often results in a PDB file.

Once you have found the PDB-file of your choosing, you can download and store this file in the `pypovray/pdb/` folder. This folder already contains a number of - pretty random - PDB files used for the included examples.

*Notes*: 
* Please store the source (URL) of the PDB file in a Readme file when handing in your work. Carefully read the BlackBoard assignment instructions; all requested files (Python code, output images/ movies, PDB files and a Readme file) must be handed in.
* Many PDB files lack a certain column of data which results in atoms not being recognized and displayed as hydrogen (default) atoms. The `pypovray` library reports on any missing element names. These element names *should* be placed at columns 77-78 (see the PDB [ATOM](http://www.wwpdb.org/documentation/file-format-content/format33/sect9.html#ATOM) description for further details).

## Placing Molecules

Once you found the molecule PDB file you'd want to visualize you can load and place it in your scene with a single line of code, just as you would with placing a `Sphere` or other Povray object. Below is a minimal example (lacking proper documentation and `main` function) of rendering a single molecule:

In [14]:
#!/usr/bin/env python3

from pypovray import pypovray, SETTINGS, models, pdb
from vapory import Scene

def frame(step):
    ''' Renders an Ethanol molecule centered in the scene '''
    ethanol = pdb.PDBMolecule('{}/pdb/ethanol.pdb'.format(SETTINGS.AppLocation), center=True)
    
    # Return the Scene object for rendering
    return Scene(models.default_camera,
                 objects=[models.default_light] + ethanol.povray_molecule)

if __name__ == '__main__':
    # Render as a single image
    pypovray.render_scene_to_png(frame, 10)

INFO:pypovray:Created a molecule from '/Users/marcelk/Development/povray/pdb/ethanol.pdb' placed at [-0.0, -0.0, 0.0] (centered is 1)


The output of this script is an image showing the Ethanol molecule centered in the scene: <img src="https://bitbucket.org/mkempenaar/pypovray/raw/master/manual/files/ethanol.png" width="350" />

Let's take a closer look at the three main modifications when comparing this code to the code of the previous example.

In [2]:
from pypovray import pypovray, SETTINGS, models, pdb

Besides the known `pypovray` and `models` modules and the `SETTINGS` object, we now also need the [`pdb` module](https://bitbucket.org/mkempenaar/pypovray/raw/master/pypovray/pdb.py). This module contains two classes that are used to construct our molecules and offers methods (instead of functions) to interact with these molecules.

In [ ]:
ethanol = pdb.PDBMolecule('{}/pdb/ethanol.pdb'.format(SETTINGS.AppLocation), center=True)

The code that actually loads a PDB file and gives us a `PDBMolecule` object which we store in the `ethanol` variable. With the call to `pdb.PDBMolecule()` we create a Python *object* that has a set of properties and methods of its own. Understanding these concepts and creating your own objects will be discussed in the next quarter (Informatics III).

The shown example provides two arguments:
* the path to the PDB file to use
    * Note that we provide the *absolute* path by prefixing the file-path with the `AppLocation`. The project already provides a number of sample PDB files in the `pdb/` folder.
* the `center` argument that will try to center the molecule around `[`<font color='red'>0</font>`,` <font color='blue'>0</font>`,` <font color='green'>0</font>`]`.
    * As you can see with the H2O molecule above, molecules are not centered by default (it deviates on the x- and z-axis).

If you leave out the `center` argument the coordinates given in the PDB file are used for placement. There is one other way to directly place a molecule by using the `offset` argument (and using the `False` value for `center`):

In [ ]:
ethanol = pdb.PDBMolecule('{}/pdb/ethanol.pdb'.format(SETTINGS.AppLocation), 
                          center=False, offset=[-10, 8, -5])

This adds the given coordinates to the coordinate of each atom and thus moves it. Do note that it is always relative to its original coordinates, i.e. if a PDB file has coordinates like `[`<font color='red'>10</font>`,` <font color='blue'>-2</font>`,` <font color='green'>5</font>`]`, using an offset of `[`<font color='red'>-5</font>`,` <font color='blue'>4</font>`,` <font color='green'>8</font>`]` would place it at `[`<font color='red'>5</font>`,` <font color='blue'>2</font>`,` <font color='green'>13</font>`]`.

At any time you can inspect your molecule to see where all the atoms are placed using the standard `print` function:

In [4]:
print(ethanol)


Overview for the molecule read from ./povray/pdb/ethanol.pdb
Idx		Atom		x	y	z
0:		C		-0.99	-0.20	-0.29	
1:		C		0.50	-0.18	0.10	
2:		H		-1.28	-1.20	-0.52	
3:		H		-1.58	0.16	0.53	
4:		H		-1.14	0.43	-1.14	
5:		H		1.10	-0.55	-0.73	
6:		H		0.66	-0.81	0.98	
7:		O		0.89	1.16	0.42	
8:		H		1.82	1.17	0.66	
Molecule is currently centered at [-0. -0.  0.]



Finally, the last code-difference is where the molecule is added to the `Scene` with:

In [ ]:
# Return the Scene object for rendering
return Scene(models.default_camera,
             objects=[models.default_light] + ethanol.povray_molecule)

Here we **add** (using the `+` symbol) a `list` of atoms to the `objects` list. This is exactly the same as we did in assignment 2 with our legend. The `ethanol` object has an attribute called `povray_molecule` which is a list containing all the atoms as Povray `Spheres` styled to look like the atoms they represent.

## Using a molecule in an animation

We have seen how to create molecules and to give them an initial position. For some molecules this might be enough (i.e. a *membrane* can be a static element) but there will most likely be molecules that we would like to animate with (move, rotate, etc.). For this purpose we are going to review our code *structure* before showing how to actually interact with a molecule. The reason is that if we create our molecule **within** our `frame` function, the data (PDB file contents) have to be read for every frame, even if we don't move it at all! If however we create the molecule once and then just use the `molecule.povray_molecule` repeatedly our program is far more efficient.

To do this, we need to create a new function that constructs our molecules that we then can use throughout our animation. There are a few downsides to the way we do this, for instance the code uses **`globals`** which are generally discouraged and might make for confusing code. But it is a tradeoff with efficiency and in this case, efficiency wins. The following code shows how to create the `ethanol` molecule in a separate function using this `global` approach:

In [15]:
#!/usr/bin/env python3

from pypovray import pypovray, SETTINGS, models, pdb
from vapory import Scene

def molecules():
    ''' Creates molecules and contains other constants '''
    global ETHANOL
    
    ETHANOL = pdb.PDBMolecule('{}/pdb/ethanol.pdb'.format(SETTINGS.AppLocation), center=True)

def frame(step):
    ''' Renders an Ethanol molecule centered in the scene '''
    
    # Return the Scene object for rendering
    return Scene(models.default_camera,
                 objects=[models.default_light] + ETHANOL.povray_molecule)

if __name__ == '__main__':
    # Create molecule(s)
    molecules()
    
    # Render as a single image
    pypovray.render_scene_to_png(frame)

INFO:pypovray:Created a molecule from '/Users/marcelk/Development/povray/pdb/ethanol.pdb' placed at [-0.0, -0.0, 0.0] (centered is 1)


There is one added function called `molecules` which creates a `global`-variable ETHANOL (using all-caps to indicate a global 'constant') and we call this function before the render command (`render_scene_to_png`). Once you have a global variable, all other functions in the code can *use* this object, but to *modify* it each function needs to add the `global ETHANOL` statement too as shown in the first line of the `molecule` function.

From now on, we will create a single function in our scripts to hold all molecules and constants (including calculated values etc.) and use the `frame` function only for performing the simulation. For a complete example regarding this structure, see the [`template_pdb.py`](https://bitbucket.org/mkempenaar/pypovray/raw/master/template_pdb.py) file (Note: this also includes a lot of code to create *help* functionality, this will not be required content, but might be interesting to study. You can get instructions on how to use this file by running `python3 template_pdb.py -h`).

## Molecule Movement

Now that we have a molecule we can interact with it, such as moving it from A to B. But, since a molecule can consist of thousands of atoms, we only specify the center of the molecule. This center is calculated so that the average distance of each atom to this center-point is the smallest. 

To move a molecule we call one of the following methods of our molecule object:
* `move_to([x, y, z])`: provided a list of x-, y- and z-coordinates, this moves the *center* of the molecule to the given location. 
* `move_offset([x, y, z])`: in some use case you want to move the molecule with an offset based on its current location. The arguments are:
    * `offset`: a coordinate list [x, y, z] indicating the distance to move on the given `axis`. To move on a single axis, set the offset of the *other* axis to 0.

Depending on how you structure your animation you might use either one of these functions. The code below shows a few examples on how to move our `ethanol` molecule:

In [ ]:
# Places the whole molecule centered at the given coordinate
ethanol.move_to([10, 5, -5]) 

# Moves the molecule 0.5-points on the x- and z-axis from the current position
ethanol.move_offset([0.5, 0, 0.5])

# Moves the molecule on all axis by the given values (2 points)
ethanol.move_offset([2, 2, 2]) 

## Rotation

Besides moving, a `PDBMolecule` object can also be *rotated*, in all directions. To rotate a molecule you need to provide at least the axis to rotate on (use your legend!) and the *angle* to rotate with, in **radians**. Rotating can be done with the `rotate` method:

    ethanol.rotate([1, 0, 0], 0.2)

where we rotate the ethanol molecule around the x-axis with `0.2` radians. To see how radians relate to degrees (and how you can convert between these two if needed), use a simple internet search. For further details and an example, see the [`template_pdb.py`](https://bitbucket.org/mkempenaar/pypovray/raw/master/template_pdb.py) file and the output on the [`pypovray`](https://bitbucket.org/mkempenaar/pypovray) repository main site, 2nd animation shown on the page.

Note that rotating a molecule is a *persistant* modification of the molecule. This means that if you rotate it by 10 degrees on the x-axis in frame **1**, it retains this position in the next frame where you can rotate it by another 10 degrees. You do not need to use the `step` variable to calculate the angle. You could however calculate the angle you want to rotate with per step.

## Splitting Molecules

Once you created a `PDBMolecule` object, you can 'split' the molecule into two `PDBMolecule` objects. This is used for instance in the ATP to ADP example code where a charged phosphate group is split off.

The `divide` method requires a list of atom indices, a name for the splitted molecule and an optional `offset` to place the splitted molecule. See line 45 in the [`atp_to_adp.py`](https://bitbucket.org/mkempenaar/pypovray/raw/master/atp_to_adp.py) example file:

In [ ]:
phosphate = ATP.divide([0, 1, 2, 3, 32, 7, 31], 'phosphate', offset=[0, -4, 0])

The list of numbers `[0, 1, 2, 3, 32, 7, 31]` refer to the atom numbers by their position in the PDB file. It might be easier to add numeric labels to the atoms and get the numbers this way, see the *Atom Labels* section below.

## Styling

Be default the styling of the molecule depends on the atoms it consists of. Each atom has its own color and size which are loaded from the [`models.py`](https://bitbucket.org/mkempenaar/pypovray/raw/master/povray/models.py) module (**NOTE:** this list is *NOT* complete, if your molecule contains atoms not defined here it will default to a Hydrogen model!).

You can however simply override the complete style of the molecule by providing a new *model* with the `model` argument when constructing the object. The code below shows the creation of a custom *model* and the construction of an ethanol molecule using this model (glass-like model)

In [ ]:
molecule_model = (Pigment('color', [1, 1, 1], 'transmit', 0.96),
                  Interior('ior', 1.05), 
                  Finish('phong', 0.5, 'reflection', 0.01))

# Create a molecule using a custom model
ethanol = pdb.PDBMolecule('{}/pdb/ethanol.pdb'.format(SETTINGS.AppLocation), center=True, model=molecule_model)

Such custom models could be used to make molecules transparent to show a 'focus' on other parts of the simulation. The image below shows the ATPase and Amylase enzymes (transparent) with their substrates; ATP and Amylopectin (colored) in their fictive active sites ([original size](https://bioinf.nl/~marcelk/pypovray/pdb_model.png)): <img src="https://bitbucket.org/mkempenaar/pypovray/raw/master/manual/files/pdb_model.png" width="650" />

With such large enzymes the rendering time is extremely long and most likely not suitable for long simulations (probably one or more hours of rendering *per frame* on our 80-core server). 

### Atom Labels

There is an experimental method that can be used to add text labels on all atoms. This could be helpful when an atom is in close view. The difficulty of the labels is that they should always face the camera, therefore we need to provide the active camera to this method. By default the atom index is shown (the (line)number in the PDB file), to show the atom name we need to set the `name` argument to `True` (see the [ATP to ADP](https://bitbucket.org/mkempenaar/pypovray/raw/master/manual/files/atp_to_adp.png) example image):

In [ ]:
# Show Atom Index (default)
ethanol.show_label(camera=models.default_camera, name=False)
# Show Atom Name
ethanol.show_label(camera=models.default_camera, name=True)

# Simulation - Program Structure

Consider a simulation that consists of four steps. Each step shows one or a few molecules moving from A to B where they interact with some other molecules. A proton pump with co-transport of sucrose (see example picture in the introduction presentation) could consist of the following steps:
* ATP moves to the proton pump
* ATP activates the proton pump and reduces to ADP
* H+ protons traverse through the pump to the extracellular space
    * (repeat until a buildup of protons are in the extracellular space creating a charge potential)
* A H+ proton and sucrose molecule traverse through the Sucrose-H+ cotransporter membrane molecule to the intracellular space

To program these steps requires somewhere between 60 and 200 lines of code (just an estimate) depending on how 'nice' you will make the movements of all elements. Until now we placed most of our code in the `scene` function which is fine for what we have done so far. But for a simulation with multiple steps such as this, putting all the code into that single function is not what we want, it will get cluttered, hard to read and very hard to maintain or change.

The solution is to first *design* your program by *identifying* parts of the program that can be seen as separate units to place them in their own functions.

Using the steps outlined above we could design the program with the following functions:
* `move_atp()`: move ATP to the proton pump
* `activate_ppump()`: Activate proton pump (conformational change) and reduce ATP
* `pump_protons()`: Pump multiple H+ protons to extracellular space
* `sucrose_cotransport()`: Diffuse sucrose and H+ protons to the intracellular space
* `scene()`: Control the simulation by calling the above listed functions in the right order and at the right moment (time- or frame-range)

Use the design techniques that are taught in the Informatics class when asked to design your program in the assignments (and see the *L1_Programming Python Flow.pptx* document in the BlackBoard *Informatica 2* course). 

# Assignment 5 - Molecular Structures

For this programming assignment we will combine the techniques discussed above. 

**Step 1**

Find a molecule of your choosing that you can split into multiple (minimum of two) separate components. This molecule can be anything you want, but the splitting should be based on biology! All [catabolic](https://en.wikipedia.org/wiki/Catabolism) reactions *break down* molecules into smaller components and these are all targets you can use for this visualization. A prime example of a splittable molecule is of course ATP which is shown here broken down into ADP and a Phosphate group: <img src="https://bitbucket.org/mkempenaar/pypovray/raw/master/manual/files/atp_to_adp.png" width="650" />

Do note however that this is not an exact representation of both resulting molecules! The conformation (exact shape) of ATP changes when it is converted into ADP which is not shown in this image.


**Step 2**

Design your program based on the following steps (note, not *all* steps are actually separate steps in your program!). This (graphical) design is a separate product to hand in with the code.


**Step 3**

Visualize the molecule in the center of your image. Depending on your molecule, change the `Camera` parameters (i.e. create a new custom `Camera` object) so that the whole molecule is visible, preferably in the middle of the `Scene`.


**Step 4**

Determine the atom-indices of the part you want to split off. You can enable the atom-labels to get an idea of where each atom is. Do note however that if you split off part of the molecule, all indices are reset; if you split off atoms `[10..20]` from the original molecule which has 40 atoms in total, you get one new molecule with atoms `[1..10]` and the original molecule is left with indices `[1..30]`. 

Disable the atom-index label if you used it, or replace it with the atom-name label before handing in the assignment.


**Step 5**

Once you have - at least - two `PDBMolecule` objects, move them apart for a small difference to show that they are actually separate molecules.


**Step 6**

Once the molecules are separated, rotate them one full circle on whatever axis you'd like.


**Tips**

Depending on the size of your molecule and if you use a `Background` and or `Plane` object to make it look more pretty; your rendering will take a long time. Please switch to the `prototype.ini` configuration file (read the [FAQ](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#I-want-to-switch-between-configuration-files;-How-do-I-do-this?)) during development and switch back to the `default.ini` before you hand in your work). Additionally, you can use one of our *servers* to perform the rendering as this will enable you to use your computer for other tasks, speedup the rendering and reduce the heat buildup in the classrooms. For intermediate rendering using the `prototype.ini` file you could use:

    ssh user@idefix

and for the final rendering:

    ssh user@assemblix


# Assignment 6 - Final Project Design

Create a design document for your final project. Discuss this with your Informatics teacher as we follow the exact same rules taught there. This document should be approved by your practical teacher before starting to work on writing your actual code. Once approved the first thing that you do is create *your own* template script that contains all the functions that you require (taken from the design). Then you'll start by filling the functions in a chronological order (as seen from the simulation) so that you can carefully test each separate step.

**Update 14/12**: A newly added feature allows you to render a specific range or number of frames into an output video. This feature was already present in the `render_scene_to_png()` function, but now the following is also possible:

    pypovray.render_scene_to_mp4(frame, range(20, 40))

This will exclusively render frames 20 through 39 and create the output MP4 video using these images. The `render_scene_to_gif` function offers the same functionality. You will be warned however if you specify frame number(s) > SETTINGS.NumberFrames.


# Assignment 7 - Materials & Methods chapter

This document concludes with writing your Materials and Methods chapter for the final report. You will require this chapter for the *Rapporteren* course soon™.

This chapter should list *all* used materials and techniques. Materials include:
* Used molecules in your final simulation
* Software (always include version numbers):
    * Python, with libraries:
        * povray
        * vapory
        * ...
    * Povray ray-tracer

The Methods part is a bit more vague at the moment, but they will be cleared up later. This part will include for instance short examples of *how* you can create, modify and visualize molecules using the software described in the section above.

Further more, you do *not* need to list your editors (geany, PyCharm, MS Word, OpenOffice, Dia, etc.) or your Operating System *unless* it is a requirement to perform your simulation (project is OS-dependent or you used certain specific editor features).
    
